In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [106]:
from typing import Optional, Union
from pydantic import Field, BaseModel

descriptions = {
    "status": """
        This field will be one of the following values:
        - Incomplete or erroneous
            Meaning: Code is incomplete or it is producing errors (it cannot be run succesfully).
        - Incorrect but better
            Meaning: Code is incorrect but better than previous code snippet in terms of syntax, logic and correctness. This means the user is moving in right direction towards the correct code.
        - Incorrect and worse
            Meaning: Code is incorrect and worse than previous code snippet in terms of syntax, logic -  This means the user is moving in wrong direction away from the correct code.
        - Incorrect and same as previous code snippet
            Meaning: Code is incorrect and same as previous code snippet and there is no improvement in the code.
        - Partially correct
            Meaning: Code is partially correct. It covers some of the requirements from the question but not all.
        - Correct but can be improved
            Meaning: Syntax is correct, code does not produce any error on running it and output is as expected BUT the code can be improved (improvements can be in terms of code optimization, better logic, better variable names, better comments, etc.)
        - Correct
            Meaning: Syntax is correct, code does not produce any error on running it and output is as expected. Code is following the guidelines provided in the question. There is no redundancy in the code. Code is optimized and well written.
        """,

    "score": """
        An approximate score out of 100 as the quality of the code snippet provided by user. This score should be based on the quality of code snippet and how well it is following the guidelines provided in the question.
    """,

    "result": """
        - Improving
            Meaning: The given code is better than the previous code snippet in terms of syntax, logic and correctness. This means the user is moving in right direction towards the correct code.
        - Not Improving
            Meaning: The given code is worse than the previous code snippet in terms of syntax, logic or correctness. This means the user is moving in wrong direction away from the correct code.
    """,

    "comment": """
        One line comment showing the reason for the given score.
    """
}


class Evaluation(BaseModel):
    """Evaluation of the code provided by the user."""

    status: str = Field(default=None, description=descriptions["status"])
    score: str = Field(default=None, description=descriptions["score"])
    result: int = Field(default=0, description=descriptions["result"])
    comment: str = Field(default=None, description=descriptions["comment"])

class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")


class FinalResponse(BaseModel):
    final_output: Union[Evaluation, ConversationalResponse]



In [107]:
model_with_structured_output = llm.with_structured_output(FinalResponse)

In [117]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage, SystemMessage
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages.ai import AIMessage

import time

workflow = StateGraph(state_schema=MessagesState)

class AgentState(MessagesState):
    messages: MessagesState
    final_output: FinalResponse

def call_model(state: AgentState) -> AgentState:
    print("STATE",state)
    response = model_with_structured_output.invoke(state["messages"])

    print(type(response))

    if isinstance(response, FinalResponse):
        print(response)
        return {"messages": BaseMessage(response.final_output)}
    else:
        print(response)
        return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

In [118]:

config = {"configurable":{"thread_id":"12355"}}

def get_code(filename:str) -> str:
    with open(filename) as f:
        s = f.read()
        return s

user_template = """
{code}
"""   
user_prompt = ChatPromptTemplate(["user",user_template])
def get_user_code_snippet() -> list[BaseMessage]:
    code_snippet = get_code("test.py")
    return user_prompt.invoke({"code": code_snippet}).to_messages()


def get_output(messages: list[BaseMessage]):
    output = app.invoke({"messages": messages}, config)
    output["messages"][-1].pretty_print()

def init():
    system_message = SystemMessage(content="""
You are a python code quality evaluator.
1. You will receive a question in next prompt.
2. Then you will receive a code snippet by user periodically. 
3. You have to evaluate the quality of code and give the output.
4. Restraint from giving the solution or suggestion or hints.
""")
    get_output([system_message])

In [119]:
init()

question = "Write a program that takes input first name and last name from the user and passes it to the function greeting_full_name to print the full name of the user with a greeting."

system_question_template = """
Questions: {question}
"""
system_question_prompt = ChatPromptTemplate(["system",system_question_template])
system_question_message = system_question_prompt.invoke({"question":question}).to_messages()
get_output(system_question_message)

STATE {'messages': [SystemMessage(content='\nYou are a python code quality evaluator.\n1. You will receive a question in next prompt.\n2. Then you will receive a code snippet by user periodically. \n3. You have to evaluate the quality of code and give the output.\n4. Restraint from giving the solution or suggestion or hints.\n', additional_kwargs={}, response_metadata={}, id='cc48346a-90c8-407a-ac90-fa83ec073779')]}
<class '__main__.FinalResponse'>
final_output=Evaluation(status='Incomplete or erroneous', score='20', result=0, comment='The code snippet is incomplete and contains errors that prevent it from running successfully.')


ValidationError: 10 validation errors for BaseMessage
content.str
  Input should be a valid string [type=string_type, input_value=Evaluation(status='Incomp... running successfully.'), input_type=Evaluation]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
content.list[union[str,dict[any,any]]].0.str
  Input should be a valid string [type=string_type, input_value=('status', 'Incomplete or erroneous'), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
content.list[union[str,dict[any,any]]].0.dict[any,any]
  Input should be a valid dictionary [type=dict_type, input_value=('status', 'Incomplete or erroneous'), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type
content.list[union[str,dict[any,any]]].1.str
  Input should be a valid string [type=string_type, input_value=('score', '20'), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.9/v

In [43]:
while True:
    user_message = get_user_code_snippet()
    get_output(user_message)
    time.sleep(5)

================================ Python Message ================================

def printName():
    print("Hello, my name is John Doe")

def printAge():
    print("I am 25 years old")
SOME ERROR Got unknown type content='def printName():\n    print("Hello, my name is John Doe")\n\ndef printAge():\n    print("I am 25 years old")' additional_kwargs={} response_metadata={} type='python' id='9bbc315b-3618-4638-8397-296e095aef58'


TypeError: SystemMessage.__init__() missing 1 required positional argument: 'content'

6